In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset 
import pyarrow as pa 
import time 
import os 
from tqdm import tqdm 
import pyarrow
from hashlib import sha256
import shutil

# Tokenize and process

1. Loop through each files in the dataset and load it into memory
2. Multi-process with all threads, then having an open stream to write into
3. Save the table down 

In [2]:
base_folder = "../data_all/data"

In [3]:
tokenizer = AutoTokenizer.from_pretrained("./trained_tokenizer/tokenizer-50k")
tokenizer.model_max_length = 10000000

In [4]:

def process_and_join(batch, num_tokens_file): 
    content = []
    token_length = 0

    hashed = sha256(str(time.time()).encode("utf-8")).hexdigest()
    stream =  pa.CompressedOutputStream(f"./temp/{hashed}.arrow", compression="gzip")
    outputs = tokenizer.batch_encode_plus(batch["text"])["input_ids"]

    for row in outputs: 
        # remove [SEP] token 
        row = row[:-1]

        if len(row) <= 128: 
            content.append(row)
            token_length += len(row)
        else: 
            for start_idx in range(0, len(row), 90): 
                token_length += 128 
                content.append(row[start_idx:start_idx+128]) 

    table = pa.Table.from_arrays(
        [pa.array(content)], names=["input_ids"])

    writer = pa.RecordBatchStreamWriter(stream, table.schema)
    writer.write_table(table)

    num_tokens_file.write(f"{token_length}\n")

    return batch

def convert_to_tokens(batch, tokenizer): 
    outputs = tokenizer.batch_encode_plus(batch["text"])["input_ids"]
    return {"input_ids": outputs}

In [5]:
# create files for status checking 
num_sentence_file = open("./num_sentence.txt", "a", encoding="utf-8") 

In [1]:
for i, file in tqdm(enumerate(sorted(os.listdir(base_folder)))): 
    if i <= 19: 
        continue 
    num_tokens_file = open(f"./num_tokens/token_{str(i).zfill(5)}.txt", "w", encoding="utf-8")

    dataset = load_dataset("../data_all/data", data_files=[file], split="train", streaming=False, cache_dir=None)
    dataset = dataset.map(process_and_join, batched=True, num_proc=14, fn_kwargs={"num_tokens_file": num_tokens_file}, batch_size=5000, remove_columns=["text"])

    num_sentence_file.write(f"{len(dataset)}\n")
    num_tokens_file.close()

    dataset = load_dataset("./temp", num_proc=16, streaming=False, split="train", cache_dir=None)
    dataset.to_parquet(f"./final/final_{str(i).zfill(5)}.parquet", batch_size=20000)

    for file in os.listdir("./temp"): 
        file_path = os.path.join("./temp", file)
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  

num_sentence_file.close()

NameError: name 'tqdm' is not defined